In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from matplotlib.gridspec import GridSpec

def load_data(json_path):
    """Load and process the planesnet JSON data."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    images = np.array(data['data'])
    labels = np.array(data['labels'])
    
    # Reshape each image into RGB channels (20x20x3)
    n_images = len(images)
    images_reshaped = []
    for i in range(n_images):
        img = images[i]
        # Split into RGB channels (each channel is 400 pixels)
        r = np.array(img[:400]).reshape(20, 20)
        g = np.array(img[400:800]).reshape(20, 20)
        b = np.array(img[800:]).reshape(20, 20)
        images_reshaped.append(np.dstack((r, g, b)))
    
    return np.array(images_reshaped), labels

def calculate_averages(images, labels):
    """Calculate average RGB values for plane and no-plane images."""
    plane_imgs = images[labels == 1]
    no_plane_imgs = images[labels == 0]
    
    plane_avg = np.mean(plane_imgs, axis=0)
    no_plane_avg = np.mean(no_plane_imgs, axis=0)
    
    return plane_avg, no_plane_avg

def plot_rgb_averages(plane_avg, no_plane_avg, output_path):
    """Create the visualization of average RGB values."""
    fig = plt.figure(figsize=(15, 10))
    gs = GridSpec(2, 3, figure=fig)
    
    titles = ['Average Red', 'Average Green', 'Average Blue']
    classes = ['Plane', 'No Plane']
    
    for i, channel in enumerate(['r', 'g', 'b']):
        # Plot for plane class
        ax1 = fig.add_subplot(gs[0, i])
        im1 = ax1.imshow(plane_avg[:,:,i], cmap='RdYlBu_r', vmin=0, vmax=255)
        plt.colorbar(im1, ax=ax1)
        ax1.set_title(f'{titles[i]} ({classes[0]})')
        
        for y in range(0, 20, 5):
            for x in range(0, 20, 5):
                avg_val = np.mean(plane_avg[y:y+5, x:x+5, i])
                ax1.text(x+2.5, y+2.5, f'{avg_val:.1f}', 
                        ha='center', va='center', fontsize=8)
        
        ax2 = fig.add_subplot(gs[1, i])
        im2 = ax2.imshow(no_plane_avg[:,:,i], cmap='RdYlBu_r', vmin=0, vmax=255)
        plt.colorbar(im2, ax=ax2)
        ax2.set_title(f'{titles[i]} ({classes[1]})')
        
        for y in range(0, 20, 5):
            for x in range(0, 20, 5):
                avg_val = np.mean(no_plane_avg[y:y+5, x:x+5, i])
                ax2.text(x+2.5, y+2.5, f'{avg_val:.1f}', 
                        ha='center', va='center', fontsize=8)
    
    plt.suptitle('Average RGB Values Over Images (Plane and No Plane)', fontsize=16)
    plt.tight_layout()
    
    plt.savefig(output_path)
    plt.close()

def main():

    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    output_path = r'C:\Users\Wu996\Desktop\Final Project\output\rgb_analysis.png'
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    print("Loading data...")
    images, labels = load_data(json_path)
    
    print("Calculating averages...")
    plane_avg, no_plane_avg = calculate_averages(images, labels)
    
    print("Creating visualization...")
    plot_rgb_averages(plane_avg, no_plane_avg, output_path)
    print(f"Visualization saved to: {output_path}")

if __name__ == "__main__":
    main()

Loading data...
Calculating averages...
Creating visualization...
Visualization saved to: C:\Users\Wu996\Desktop\Final Project\output\rgb_analysis.png


Classic Machine Learning

In [3]:
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import time

def load_and_preprocess_data(json_path):
    """Load and preprocess the planesnet data."""
    print("Loading data...")
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    X = np.array(data['data'])
    y = np.array(data['labels'])
    
    X = X / 255.0
    
    return X, y

def create_simple_nn():
    """Create a simple neural network."""
    model = models.Sequential([
        layers.Dense(100, activation='relu', input_shape=(1200,)),
        layers.Dropout(0.3),
        layers.Dense(50, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(y_true, y_pred):
    """Calculate various metrics for model evaluation."""
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred)
    }

def train_and_evaluate_models(X, y):
    """Train and evaluate multiple models."""
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Define models to test
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=100, n_jobs=-1),
        'XGBoost': xgb.XGBClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            n_jobs=-1,
            tree_method='hist'
        ),
        'LightGBM': lgb.LGBMClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            n_jobs=-1
        ),
        'SVM': SVC(kernel='rbf'),
        'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
        'Decision Tree': DecisionTreeClassifier(),
        'Naive Bayes': GaussianNB()
    }
    
    # Add neural network
    models['Simple NN'] = create_simple_nn()
    
    results = {}
    training_times = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        start_time = time.time()
        
        if name == 'Simple NN':
            history = model.fit(X_train, y_train, 
                              epochs=10, 
                              batch_size=32, 
                              validation_split=0.2,
                              verbose=0)
            y_pred = (model.predict(X_test) > 0.5).astype(int)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        training_time = time.time() - start_time
        
        metrics = evaluate_model(y_test, y_pred)
        results[name] = metrics
        training_times[name] = training_time
        
        print(f"{name} Results:")
        print(f"Training Time: {training_time:.2f} seconds")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"Precision: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1-Score: {metrics['f1']:.4f}")
    
    return results, training_times

def plot_results(results, training_times, output_path):
    """Create visualization of model performance."""
    plt.figure(figsize=(15, 12))
    
    plt.subplot(2, 1, 1)
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    x = np.arange(len(results))
    width = 0.2
    colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c']
    
    for i, (metric, color) in enumerate(zip(metrics, colors)):
        values = [results[model][metric] for model in results]
        plt.bar(x + i*width, values, width, label=metric.capitalize(), color=color, alpha=0.7)
    
    plt.xlabel('Models')
    plt.ylabel('Score')
    plt.title('Model Performance Comparison')
    plt.xticks(x + width*1.5, list(results.keys()), rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 1, 2)
    plt.bar(training_times.keys(), training_times.values(), color='#f39c12', alpha=0.7)
    plt.xlabel('Models')
    plt.ylabel('Training Time (seconds)')
    plt.title('Model Training Times')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

def main():
    # Set paths
    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    output_path = r'C:\Users\Wu996\Desktop\Final Project\output\model_comparison_light.png'
    
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Load data
    X, y = load_and_preprocess_data(json_path)
    
    results, training_times = train_and_evaluate_models(X, y)
    
    plot_results(results, training_times, output_path)
    
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    for metric in metrics:
        best_model = max(results.items(), key=lambda x: x[1][metric])
        print(f"\nBest Model by {metric}: {best_model[0]}")
        print(f"{metric}: {best_model[1][metric]:.4f}")
    
    fastest_model = min(training_times.items(), key=lambda x: x[1])
    print(f"\nFastest Model: {fastest_model[0]}")
    print(f"Training Time: {fastest_model[1]:.2f} seconds")

if __name__ == "__main__":
    main()

d:\python-3.12.4-amd64\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Loading data...


d:\python-3.12.4-amd64\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training Logistic Regression...
Logistic Regression Results:
Training Time: 4.51 seconds
Accuracy: 0.9080
Precision: 0.8168
Recall: 0.8132
F1-Score: 0.8150

Training Random Forest...
Random Forest Results:
Training Time: 7.75 seconds
Accuracy: 0.9508
Precision: 0.9301
Recall: 0.8677
F1-Score: 0.8978

Training XGBoost...
XGBoost Results:
Training Time: 12.51 seconds
Accuracy: 0.9573
Precision: 0.9226
Recall: 0.9047
F1-Score: 0.9136

Training LightGBM...
[LightGBM] [Info] Number of positive: 6405, number of negative: 19195
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.166068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 290985
[LightGBM] [Info] Number of data points in the train set: 25600, number of used features: 1200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.250195 -> initscore=-1.097571
[LightGBM] [Info] Start training from score -1.097571
[LightGBM] [Warning] No further splits with p

d:\python-3.12.4-amd64\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Best Model by accuracy: XGBoost
accuracy: 0.9573

Best Model by precision: Random Forest
precision: 0.9301

Best Model by recall: KNN
recall: 0.9160

Best Model by f1: XGBoost
f1: 0.9136

Fastest Model: Naive Bayes
Training Time: 0.43 seconds


CNN & ResNet

In [2]:
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2

def load_data(json_path):
    """Load and preprocess the planesnet data."""
    print("Loading data...")
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    X = np.array(data['data'])
    y = np.array(data['labels'])
    
    # Reshape images to 20x20x3
    n_images = len(X)
    X_reshaped = np.zeros((n_images, 20, 20, 3))
    for i in range(n_images):
        r = X[i][:400].reshape(20, 20)
        g = X[i][400:800].reshape(20, 20)
        b = X[i][800:].reshape(20, 20)
        X_reshaped[i] = np.dstack((r, g, b))
    
    X_reshaped = X_reshaped / 255.0
    
    return X_reshaped, y

def create_basic_cnn():
    """Create a basic CNN model."""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model, "Basic CNN"

def create_deep_cnn():
    """Create a deeper CNN model."""
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(20, 20, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model, "Deep CNN"

def create_original_model():
    """Create the original model from the paper."""
    model = models.Sequential([
        layers.Dense(300, activation='relu', input_shape=(1200,)),
        layers.Dense(100, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model, "Original Model"

def create_residual_block(x, filters, kernel_size=3):
    """Create a residual block."""
    y = layers.Conv2D(filters, kernel_size, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Conv2D(filters, kernel_size, padding='same')(y)
    y = layers.BatchNormalization()(y)
    
    if x.shape[-1] != filters:
        x = layers.Conv2D(filters, 1, padding='same')(x)
    
    out = layers.Add()([x, y])
    out = layers.Activation('relu')(out)
    return out

def create_custom_resnet():
    """Create a custom ResNet-style model."""
    inputs = layers.Input(shape=(20, 20, 3))
    
    x = layers.Conv2D(32, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = create_residual_block(x, 32)
    x = create_residual_block(x, 32)
    x = layers.MaxPooling2D()(x)
    
    x = create_residual_block(x, 64)
    x = create_residual_block(x, 64)
    x = layers.GlobalAveragePooling2D()(x)
    
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    return model, "Custom ResNet"

def train_and_evaluate_models(X, y, output_path):
    """Train and evaluate multiple model architectures."""

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model_creators = [
        create_basic_cnn,
        create_deep_cnn,
        create_custom_resnet
    ]
    
    results = []
    
    for create_model in model_creators:
        model, model_name = create_model()
        print(f"\nTraining {model_name}...")
        
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        history = model.fit(
            X_train, y_train,
            epochs=20,
            batch_size=32,
            validation_split=0.2,
            verbose=1
        )
        
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        results.append({
            'name': model_name,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'history': history.history
        })
        
        print(f"{model_name} Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
    
    plot_results(results, output_path)
    
    best_model = max(results, key=lambda x: x['accuracy'])
    print(f"\nBest Model: {best_model['name']}")
    print(f"Best Accuracy: {best_model['accuracy']:.4f}")
    
    return results

def plot_results(results, output_path):
    """Create visualization of model performance."""
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 1, 1)
    metrics = ['accuracy', 'precision', 'recall', 'f1']
    x = np.arange(len(results))
    width = 0.2
    
    for i, metric in enumerate(metrics):
        values = [r[metric] for r in results]
        plt.bar(x + i*width, values, width, label=metric.capitalize())
    
    plt.xlabel('Models')
    plt.ylabel('Score')
    plt.title('Model Performance Comparison')
    plt.xticks(x + width*1.5, [r['name'] for r in results])
    plt.legend()
    
    plt.subplot(2, 1, 2)
    for result in results:
        plt.plot(result['history']['val_accuracy'], label=f"{result['name']}")
    
    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    plt.title('Training History')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def main():
    # Set paths
    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    output_path = r'C:\Users\Wu996\Desktop\Final Project\output\model_comparison.png'
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Load data
    X, y = load_data(json_path)
    
    # Train and evaluate models
    results = train_and_evaluate_models(X, y, output_path)

if __name__ == "__main__":
    main()

Loading data...

Training Basic CNN...
Epoch 1/20


d:\python-3.12.4-amd64\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


640/640 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8077 - loss: 0.4313 - val_accuracy: 0.9402 - val_loss: 0.1576
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9385 - loss: 0.1554 - val_accuracy: 0.9537 - val_loss: 0.1251
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9474 - loss: 0.1319 - val_accuracy: 0.9535 - val_loss: 0.1225
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9523 - loss: 0.1242 - val_accuracy: 0.9645 - val_loss: 0.0991
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9624 - loss: 0.1006 - val_accuracy: 0.9557 - val_loss: 0.1204
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9626 - loss: 0.0952 - val_accuracy: 0.9611 - val_loss: 0.1065
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9702 - loss: 0.0821 - val_accuracy: 0.9678 - val_loss: 0.0926
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9719 - loss: 0.0769 - val_accuracy: 0.9662 - val_

d:\python-3.12.4-amd64\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


640/640 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.8594 - loss: 0.3345 - val_accuracy: 0.8900 - val_loss: 0.2282
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.9467 - loss: 0.1382 - val_accuracy: 0.8881 - val_loss: 0.2924
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.9612 - loss: 0.0953 - val_accuracy: 0.9697 - val_loss: 0.0782
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9688 - loss: 0.0830 - val_accuracy: 0.9715 - val_loss: 0.0806
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.9748 - loss: 0.0697 - val_accuracy: 0.9367 - val_loss: 0.1695
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.9770 - loss: 0.0615 - val_accuracy: 0.9516 - val_loss: 0.1268
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9813 - loss: 0.0559 - val_accuracy: 0.9824 - val_loss: 0.0558
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9808 - loss: 0.0535 - val_accurac

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, average_precision_score
import json
from sklearn.model_selection import train_test_split
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from sklearn.metrics import auc
import shap
import numpy as np
from tensorflow.keras.models import load_model
import cv2

def create_residual_block(x, filters, kernel_size=3):
    """Create a residual block."""
    y = layers.Conv2D(filters, kernel_size, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Conv2D(filters, kernel_size, padding='same')(y)
    y = layers.BatchNormalization()(y)
    
    if x.shape[-1] != filters:
        x = layers.Conv2D(filters, 1, padding='same')(x)
    
    out = layers.Add()([x, y])
    out = layers.Activation('relu')(out)
    return out

def create_custom_resnet():
    """Create the Custom ResNet model."""
    inputs = layers.Input(shape=(20, 20, 3))
    
    x = layers.Conv2D(32, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = create_residual_block(x, 32)
    x = create_residual_block(x, 32)
    x = layers.MaxPooling2D()(x)
    
    x = create_residual_block(x, 64)
    x = create_residual_block(x, 64)
    x = layers.GlobalAveragePooling2D()(x)
    
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    return model

def load_data(json_path):
    """Load and preprocess the planesnet data."""
    print("Loading data...")
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Convert to numpy arrays
    X = np.array(data['data'])
    y = np.array(data['labels'])
    
    # Reshape images to 20x20x3
    n_images = len(X)
    X_reshaped = np.zeros((n_images, 20, 20, 3))
    for i in range(n_images):
        r = X[i][:400].reshape(20, 20)
        g = X[i][400:800].reshape(20, 20)
        b = X[i][800:].reshape(20, 20)
        X_reshaped[i] = np.dstack((r, g, b))
    
    # Normalize pixel values
    X_reshaped = X_reshaped / 255.0
    
    return X_reshaped, y

def plot_training_history(history, output_path):
    """Create an interactive training history plot."""
    fig = make_subplots(rows=1, cols=2, 
                       subplot_titles=('Model Accuracy', 'Model Loss'))
    
    # Accuracy subplot
    fig.add_trace(
        go.Scatter(y=history.history['accuracy'], name="Training Accuracy",
                  line=dict(color='royalblue', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_accuracy'], name="Validation Accuracy",
                  line=dict(color='lightblue', width=2, dash='dash')),
        row=1, col=1
    )
    
    # Loss subplot
    fig.add_trace(
        go.Scatter(y=history.history['loss'], name="Training Loss",
                  line=dict(color='firebrick', width=2)),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_loss'], name="Validation Loss",
                  line=dict(color='lightcoral', width=2, dash='dash')),
        row=1, col=2
    )
    
    fig.update_layout(height=500, width=1000, title_text="Training History")
    fig.write_html(os.path.join(output_path, 'training_history.html'))

def plot_confusion_matrix(y_true, y_pred, output_path):
    """Create an enhanced confusion matrix visualization."""
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate percentages
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Create annotation text
    annotations = []
    for i in range(2):
        for j in range(2):
            annotations.append(
                f'Count: {cm[i, j]}<br>Percentage: {cm_percent[i, j]:.1f}%'
            )
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=cm,
        x=['Predicted Negative', 'Predicted Positive'],
        y=['Actual Negative', 'Actual Positive'],
        text=annotations,
        texttemplate="%{text}",
        textfont={"size": 12},
        colorscale='RdBu',
    ))
    
    fig.update_layout(
        title='Confusion Matrix',
        xaxis_title='Predicted Label',
        yaxis_title='True Label',
        width=800,
        height=800
    )
    
    fig.write_html(os.path.join(output_path, 'confusion_matrix.html'))

def plot_roc_pr_curves(y_true, y_pred_proba, output_path):
    """Create interactive ROC and PR curves."""
    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Calculate PR curve
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = average_precision_score(y_true, y_pred_proba)
    
    # Create subplot
    fig = make_subplots(rows=1, cols=2, 
                       subplot_titles=('ROC Curve', 'Precision-Recall Curve'))
    
    # ROC curve
    fig.add_trace(
        go.Scatter(x=fpr, y=tpr, name=f'ROC Curve (AUC = {roc_auc:.3f})',
                  fill='tozeroy', line=dict(color='royalblue', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1], name='Random Classifier',
                  line=dict(color='gray', dash='dash')),
        row=1, col=1
    )
    
    # PR curve
    fig.add_trace(
        go.Scatter(x=recall, y=precision, 
                  name=f'PR Curve (AP = {pr_auc:.3f})',
                  fill='tozeroy', line=dict(color='firebrick', width=2)),
        row=1, col=2
    )
    
    fig.update_layout(
        height=500, width=1000,
        showlegend=True,
        title_text="Model Performance Curves"
    )
    
    fig.write_html(os.path.join(output_path, 'performance_curves.html'))

def visualize_feature_maps(model, X_sample, output_path):
    """Visualize feature maps from different layers."""
    # Get feature maps from intermediate layers
    layer_outputs = [layer.output for layer in model.layers if isinstance(layer, layers.Conv2D)]
    feature_model = models.Model(inputs=model.input, outputs=layer_outputs)
    
    # Get feature maps for a sample image
    feature_maps = feature_model.predict(X_sample[np.newaxis, ...])
    
    # Create visualization for each conv layer
    for layer_idx, feature_map in enumerate(feature_maps):
        # Create grid of feature maps
        n_features = min(8, feature_map.shape[-1])  # Display up to 8 features
        size = feature_map.shape[1]
        display_grid = np.zeros((size * 2, size * 4))
        
        for i in range(2):
            for j in range(4):
                if i * 4 + j < n_features:
                    display_grid[i * size:(i + 1) * size, 
                               j * size:(j + 1) * size] = feature_map[0, :, :, i * 4 + j]
        
        # Normalize the grid
        display_grid = (display_grid - display_grid.min()) / (display_grid.max() - display_grid.min())
        
        plt.figure(figsize=(10, 5))
        plt.title(f'Feature Maps - Conv Layer {layer_idx + 1}')
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        plt.colorbar()
        plt.savefig(os.path.join(output_path, f'feature_maps_layer_{layer_idx + 1}.png'))
        plt.close()

def main():
    # Set paths
    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    output_path = r'C:\Users\Wu996\Desktop\Final Project\output'
    model_path = r'C:\Users\Wu996\Desktop\Final Project\models\custom_resnet.h5'
    
    # Create output directories
    os.makedirs(output_path, exist_ok=True)
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    
    # Load and preprocess data
    X, y = load_data(json_path)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Create and train model
    model = create_custom_resnet()
    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    history = model.fit(X_train, y_train,
                       epochs=20,
                       batch_size=32,
                       validation_split=0.2,
                       verbose=1)
    
    # Save the model
    model.save(model_path)
    
    # Generate predictions
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Create visualizations
    print("Creating visualizations...")
    
    # 1. Training history
    plot_training_history(history, output_path)
    
    # 2. Confusion matrix
    plot_confusion_matrix(y_test, y_pred, output_path)
    
    # 3. ROC and PR curves
    plot_roc_pr_curves(y_test, y_pred_proba, output_path)
    
    # 4. Feature maps visualization
    visualize_feature_maps(model, X_test[0], output_path)
    
    print(f"Model and visualizations saved to {output_path}")

if __name__ == "__main__":
    main()

Loading data...
Epoch 1/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.8910 - loss: 0.2509 - val_accuracy: 0.8846 - val_loss: 0.2884
Epoch 2/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9614 - loss: 0.0999 - val_accuracy: 0.7777 - val_loss: 0.7223
Epoch 3/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 24s 37ms/step - accuracy: 0.9759 - loss: 0.0674 - val_accuracy: 0.9686 - val_loss: 0.0882
Epoch 4/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.9784 - loss: 0.0613 - val_accuracy: 0.9760 - val_loss: 0.0642
Epoch 5/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 26s 40ms/step - accuracy: 0.9826 - loss: 0.0487 - val_accuracy: 0.7563 - val_loss: 2.5571
Epoch 6/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.9832 - loss: 0.0467 - val_accuracy: 0.9703 - val_loss: 0.0878
Epoch 7/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 49ms/step - accuracy: 0.9867 - loss: 0.0381 - val_accuracy: 0.8053 - val_loss: 0.8472
Epoch 8/20
640/640 ━━━━━━━━━━━━━━━━━━━━ 32s 49ms/step - accuracy: 0.9879 -

200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Creating visualizations...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
Model and visualizations saved to C:\Users\Wu996\Desktop\Final Project\output


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import json
import os

def load_and_preprocess_data(json_path):
    """Load and preprocess the planesnet data."""
    print("Loading data...")
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Convert to numpy arrays
    X = np.array(data['data'])
    y = np.array(data['labels'])
    
    # Reshape images to 20x20x3
    n_images = len(X)
    X_reshaped = np.zeros((n_images, 20, 20, 3))
    for i in range(n_images):
        r = X[i][:400].reshape(20, 20)
        g = X[i][400:800].reshape(20, 20)
        b = X[i][800:].reshape(20, 20)
        X_reshaped[i] = np.dstack((r, g, b))
    
    # Normalize pixel values
    X_reshaped = X_reshaped / 255.0
    
    return X_reshaped, y

def create_enhanced_confusion_matrix(y_true, y_pred, output_path):
    """Create an enhanced confusion matrix visualization with direct number display."""
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate percentages
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Create figure and axes
    plt.figure(figsize=(12, 10))
    
    # Create heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Predicted\nNegative', 'Predicted\nPositive'],
                yticklabels=['Actual\nNegative', 'Actual\nPositive'])
    
    # Add percentage values
    for i in range(2):
        for j in range(2):
            plt.text(j + 0.5, i + 0.7, f'({cm_percent[i, j]:.1f}%)',
                    ha='center', va='center',
                    color='black' if cm_percent[i, j] < 70 else 'white')
    
    # Customize the plot
    plt.title('Confusion Matrix', pad=20, fontsize=16, fontweight='bold')
    
    # Add a custom legend/explanation
    plt.figtext(0.15, -0.05, 
                'Format: count (percentage)\nPercentages calculated row-wise (per actual class)',
                ha='left', fontsize=10, style='italic')
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

def main():
    # Set paths
    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    model_path = r'C:\Users\Wu996\Desktop\Final Project\models\custom_resnet.h5'
    output_path = r'C:\Users\Wu996\Desktop\Final Project\output\confusion_matrix_enhanced.png'
    
    # Create output directory if doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Load data
    X, y = load_and_preprocess_data(json_path)
    
    # Load the saved model
    print("Loading model...")
    model = load_model(model_path)
    
    # Generate predictions
    print("Generating predictions...")
    y_pred_proba = model.predict(X)
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Create and save confusion matrix visualization
    print("Creating confusion matrix visualization...")
    create_enhanced_confusion_matrix(y, y_pred, output_path)
    print(f"Visualization saved to: {output_path}")

if __name__ == "__main__":
    main()

Loading data...


Loading model...
Generating predictions...
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step
Creating confusion matrix visualization...
Visualization saved to: C:\Users\Wu996\Desktop\Final Project\output\confusion_matrix_enhanced.png


scenes

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.models import load_model
import json
from tqdm import tqdm

def create_basic_cnn():
    """Create a basic CNN model."""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def load_and_preprocess_data(json_path):
    """Load and preprocess the planesnet data."""
    print("Loading training data...")
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    X = np.array(data['data'])
    y = np.array(data['labels'])
    
    # Reshape images to 20x20x3
    n_images = len(X)
    X_reshaped = np.zeros((n_images, 20, 20, 3))
    for i in range(n_images):
        r = X[i][:400].reshape(20, 20)
        g = X[i][400:800].reshape(20, 20)
        b = X[i][800:].reshape(20, 20)
        X_reshaped[i] = np.dstack((r, g, b))
    
    # Normalize pixel values
    X_reshaped = X_reshaped / 255.0
    
    return X_reshaped, y

def batch_process_scene(model, scene, batch_size=64, step_size=5): 
    """Process scene in batches for faster detection."""
    height, width = scene.shape[:2]
    windows = []
    positions = []
    
    # Extract all windows and their positions with smaller step size
    for y in range(0, height - 20, step_size):
        for x in range(0, width - 20, step_size):
            window = scene[y:y+20, x:x+20]
            if window.shape[:2] == (20, 20):
                windows.append(window)
                positions.append((x, y))
    
    # Convert to numpy array and normalize
    windows = np.array(windows).astype('float32') / 255.0
    
    # Batch predict
    detections = []
    for i in range(0, len(windows), batch_size):
        batch = windows[i:i + batch_size]
        predictions = model.predict(batch, verbose=0)
        
        # Add detections above threshold
        for j, pred in enumerate(predictions):
            if pred[0] > 0.3: 
                x, y = positions[i + j]
                detections.append((x, y, float(pred[0])))
    
    return detections

def non_max_suppression(boxes, overlap_thresh=0.2): 
    """Apply non-maximum suppression."""
    if not boxes:
        return []
    
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = x1 + 20
    y2 = y1 + 20
    scores = boxes[:, 2]
    
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]
    
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        overlap = (w * h) / areas[order[1:]]
        
        inds = np.where(overlap <= overlap_thresh)[0]
        order = order[inds + 1]
    
    return boxes[keep].tolist()

def visualize_results(scene, detections, title, save_path):
    """Visualize detection results."""
    plt.figure(figsize=(20, 15))
    plt.imshow(scene)
    
    # Sort detections by confidence
    detections = sorted(detections, key=lambda x: x[2], reverse=True)
    
    # Draw detection boxes with different colors based on confidence
    for x, y, conf in detections:
        # Color mapping based on confidence (red for high confidence, yellow for lower)
        color = plt.cm.RdYlBu_r(conf)
        rect = plt.Rectangle((x, y), 20, 20, fill=False, color=color, linewidth=2)
        plt.gca().add_patch(rect)
        plt.text(x, y-5, f'{conf:.2f}', color='black', 
                bbox=dict(facecolor='white', alpha=0.7))
    
    plt.title(f'{title}\nTotal Detections: {len(detections)}')
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

def main():
    # Set paths
    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    scenes_dir = r'C:\Users\Wu996\Desktop\Final Project\archive\scenes\scenes'
    output_dir = r'C:\Users\Wu996\Desktop\Final Project\output\scene_verification'
    model_path = r'C:\Users\Wu996\Desktop\Final Project\models\custom_resnet.h5'
    basic_cnn_path = r'C:\Users\Wu996\Desktop\Final Project\models\basic_cnn.h5'
    
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.dirname(basic_cnn_path), exist_ok=True)
    
    X_train, y_train = load_and_preprocess_data(json_path)
    
    if not os.path.exists(basic_cnn_path):
        print("Training Basic CNN...")
        basic_cnn = create_basic_cnn()
        basic_cnn.compile(optimizer='adam',
                         loss='binary_crossentropy',
                         metrics=['accuracy'])
        
        basic_cnn.fit(X_train, y_train,
                     epochs=20,
                     batch_size=40,
                     validation_split=0.2,
                     verbose=1)
        
        basic_cnn.save(basic_cnn_path)
    else:
        print("Loading trained Basic CNN...")
        basic_cnn = load_model(basic_cnn_path)
    
    print("Loading Custom ResNet...")
    custom_resnet = load_model(model_path)
    
    scene_files = [f for f in os.listdir(scenes_dir) if f.endswith('.png')]
    
    for scene_file in scene_files:
        print(f"\nProcessing {scene_file}...")
        scene_path = os.path.join(scenes_dir, scene_file)
        
        scene = cv2.imread(scene_path)
        scene = cv2.cvtColor(scene, cv2.COLOR_BGR2RGB)
        
        for model, model_name in [(basic_cnn, "Basic CNN"), (custom_resnet, "Custom ResNet")]:
            print(f"Running {model_name} detection...")
            detections = batch_process_scene(model, scene, step_size=5) 
            detections = non_max_suppression(detections, overlap_thresh=0.2) 
            
            visualize_results(
                scene,
                detections,
                f"{model_name} Detection - {scene_file}",
                os.path.join(output_dir, f'{model_name.lower().replace(" ", "_")}_{scene_file}')
            )
    
    print("\nProcessing complete!")

if __name__ == "__main__":
    main()

Loading training data...


Loading trained Basic CNN...


Loading Custom ResNet...

Processing scene_1.png...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing scene_2.png...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing scene_3.png...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing scene_4.png...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing complete!


scenes_1

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.models import load_model
import json
from tqdm import tqdm

def create_basic_cnn():
    """Create a basic CNN model."""
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def load_and_preprocess_data(json_path):
    """Load and preprocess the planesnet data."""
    print("Loading training data...")
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    X = np.array(data['data'])
    y = np.array(data['labels'])
    
    # Reshape images to 20x20x3
    n_images = len(X)
    X_reshaped = np.zeros((n_images, 20, 20, 3))
    for i in range(n_images):
        r = X[i][:400].reshape(20, 20)
        g = X[i][400:800].reshape(20, 20)
        b = X[i][800:].reshape(20, 20)
        X_reshaped[i] = np.dstack((r, g, b))
    
    # Normalize pixel values
    X_reshaped = X_reshaped / 255.0
    
    return X_reshaped, y

def batch_process_scene(model, scene, batch_size=64, step_size=2):  #step_size!
    """Process scene in batches for faster detection."""
    height, width = scene.shape[:2]
    windows = []
    positions = []
    
    # Extract all windows and their positions with smaller step size
    for y in range(0, height - 20, step_size):
        for x in range(0, width - 20, step_size):
            window = scene[y:y+20, x:x+20]
            if window.shape[:2] == (20, 20):
                windows.append(window)
                positions.append((x, y))
    
    # Convert to numpy array and normalize
    windows = np.array(windows).astype('float32') / 255.0
    
    # Batch predict
    detections = []
    for i in range(0, len(windows), batch_size):
        batch = windows[i:i + batch_size]
        predictions = model.predict(batch, verbose=0)
        
        # Add detections above threshold
        for j, pred in enumerate(predictions):
            if pred[0] > 0.3: 
                x, y = positions[i + j]
                detections.append((x, y, float(pred[0])))
    
    return detections

def non_max_suppression(boxes, overlap_thresh=0.2): 
    """Apply non-maximum suppression."""
    if not boxes:
        return []
    
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = x1 + 20
    y2 = y1 + 20
    scores = boxes[:, 2]
    
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]
    
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        overlap = (w * h) / areas[order[1:]]
        
        inds = np.where(overlap <= overlap_thresh)[0]
        order = order[inds + 1]
    
    return boxes[keep].tolist()

def visualize_results(scene, detections, title, save_path):
    """Visualize detection results."""
    plt.figure(figsize=(20, 15))
    plt.imshow(scene)
    
    # Sort detections by confidence
    detections = sorted(detections, key=lambda x: x[2], reverse=True)
    
    # Draw detection boxes with different colors based on confidence
    for x, y, conf in detections:
        # Color mapping based on confidence (red for high confidence, yellow for lower)
        color = plt.cm.RdYlBu_r(conf)
        rect = plt.Rectangle((x, y), 20, 20, fill=False, color=color, linewidth=2)
        plt.gca().add_patch(rect)
        plt.text(x, y-5, f'{conf:.2f}', color='black', 
                bbox=dict(facecolor='white', alpha=0.7))
    
    plt.title(f'{title}\nTotal Detections: {len(detections)}')
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

def main():
    # Set paths
    json_path = r'C:\Users\Wu996\Desktop\Final Project\archive\planesnet.json'
    scenes_dir = r'C:\Users\Wu996\Desktop\Final Project\archive\scenes\scenes_1'
    output_dir = r'C:\Users\Wu996\Desktop\Final Project\output\scene_verification_1'
    model_path = r'C:\Users\Wu996\Desktop\Final Project\models\custom_resnet.h5'
    basic_cnn_path = r'C:\Users\Wu996\Desktop\Final Project\models\basic_cnn.h5'
    
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.dirname(basic_cnn_path), exist_ok=True)
    
    X_train, y_train = load_and_preprocess_data(json_path)
    
    if not os.path.exists(basic_cnn_path):
        print("Training Basic CNN...")
        basic_cnn = create_basic_cnn()
        basic_cnn.compile(optimizer='adam',
                         loss='binary_crossentropy',
                         metrics=['accuracy'])
        
        basic_cnn.fit(X_train, y_train,
                     epochs=20,
                     batch_size=40,
                     validation_split=0.2,
                     verbose=1)
        
        basic_cnn.save(basic_cnn_path)
    else:
        print("Loading trained Basic CNN...")
        basic_cnn = load_model(basic_cnn_path)
    
    print("Loading Custom ResNet...")
    custom_resnet = load_model(model_path)
    
    scene_files = [f for f in os.listdir(scenes_dir) if f.endswith('.jpg')]
    
    for scene_file in scene_files:
        print(f"\nProcessing {scene_file}...")
        scene_path = os.path.join(scenes_dir, scene_file)
        
        scene = cv2.imread(scene_path)
        scene = cv2.cvtColor(scene, cv2.COLOR_BGR2RGB)
        
        for model, model_name in [(basic_cnn, "Basic CNN"), (custom_resnet, "Custom ResNet")]:
            print(f"Running {model_name} detection...")
            detections = batch_process_scene(model, scene, step_size=5) 
            detections = non_max_suppression(detections, overlap_thresh=0.2) 
            
            visualize_results(
                scene,
                detections,
                f"{model_name} Detection - {scene_file}",
                os.path.join(output_dir, f'{model_name.lower().replace(" ", "_")}_{scene_file}')
            )
    
    print("\nProcessing complete!")

if __name__ == "__main__":
    main()

Loading training data...


Loading trained Basic CNN...
Loading Custom ResNet...

Processing images.jpg...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing images1.jpg...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing images2.jpg...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing images3.jpg...
Running Basic CNN detection...
Running Custom ResNet detection...

Processing complete!
